In [1]:
!unzip topostext_ner_model.zip

Archive:  topostext_ner_model.zip
   creating: content/models/model-best/
  inflating: content/models/model-best/config.cfg  
  inflating: content/models/model-best/meta.json  
   creating: content/models/model-best/vocab/
 extracting: content/models/model-best/vocab/lookups.bin  
 extracting: content/models/model-best/vocab/key2row  
  inflating: content/models/model-best/vocab/strings.json  
 extracting: content/models/model-best/vocab/vectors.cfg  
  inflating: content/models/model-best/vocab/vectors  
   creating: content/models/model-best/ner/
  inflating: content/models/model-best/ner/moves  
  inflating: content/models/model-best/ner/model  
  inflating: content/models/model-best/ner/cfg  
  inflating: content/models/model-best/tokenizer  
   creating: content/models/model-best/tok2vec/
  inflating: content/models/model-best/tok2vec/model  
 extracting: content/models/model-best/tok2vec/cfg  


In [5]:
import spacy
trained_nlp = spacy.load("/content/content/models/model-best") ## not sure why unzipping gives a 'content' folder, but this should work

In [213]:
## getting some interesting text
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.gutenberg.org/files/830/830-h/830-h.htm' # Apollonius Rhodius' Argonautica
soup = BeautifulSoup(requests.get(url).text)

arg_dict = {}
for i,p in enumerate(soup.find_all('p')[12:-78]):
  arg_dict[i] = p.text

arg = pd.DataFrame.from_dict(arg_dict,orient='index').rename(columns={0:'text'})

In [233]:
import random
rand_idx = random.randrange(0,len(arg))
text = arg.text.iloc[rand_idx] ## get random text
print(f'{rand_idx}/{len(arg)}', text, sep='\n')

241/426

“After all then, ye were not destined to leave me in your heedlessness
and to wander far; but fate has turned you back. Poor wretch that I am! What a
yearning for Hellas from some woeful madness seized you at the behest of your
father Phrixus. Bitter sorrows for my heart did he ordain when dying. And why
should ye go to the city of Orchomenus, whoever this Orchomenus is, for the
sake of Athamas’ wealth, leaving your mother alone to bear her
grief?”



In [234]:
doc = trained_nlp(text)

In [235]:
places = []
for ent in doc.ents:
  if ent.label_ == 'place':
    places.append(ent) 
    
print(len(places), places, sep='\t') ## if this is empty then the random text didn't have any places

2	[Orchomenus, Orchomenus]


In [236]:
na = pd.read_csv('/content/names.csv') ## place name look up
pl = pd.read_csv('/content/places.csv') ## coord look up

In [237]:
import folium
map = folium.Map((40,17), zoom_start=6)## should start at the average of all point, but just use this for the time being
for place in places:
  try:
    id = na.loc[na['id'] == place.text.lower()].sort_values('year_after_which').iloc[0].place_id ## getting the oldest, naive disambiguation
    lat = pl.loc[pl['id'] == id]['representative_latitude'].iloc[0]
    lon = pl.loc[pl['id'] == id]['representative_longitude'].iloc[0]
    folium.Marker((lat,lon)).add_to(map)
  except IndexError:
    print(f'{place} is not listed in Pleiades. Likely, it is not a placename.')
map

# Further work:
* consider different disambiguation technqiues
* center map on coordinates
* play with training data a bit
* integration into reading environment